In [ ]:
import os
import json

import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression

from utils.func import read_data, softmax
from utils.metric import evaluate

In [ ]:
model_name = "LLaVA-7B"
prompt = "oe"

In [ ]:
_, x_train, y_train = read_data(model_name, "MAD", split="train", 
                                prompt=prompt, token_idx=0)
val_data, x_val, y_val = read_data(model_name, "MAD", split="val", 
                                   prompt=prompt, token_idx=0, return_data=True)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

### Evaluate the original performance of LVLMs

In [ ]:
label_list = [ins['label'] for ins in val_data]
    
if prompt == "oe":
    val_labeled = json.load(open(f'./output/{model_name}/MAD_val_oe_labeled.json'))
    pred_list = [0 if ins["is_answer"] == 'no' else 1 for ins in val_labeled]
elif prompt == "oeh":
    pred_list = [0 if ins['response'].startswith("Sorry, I cannot answer your question") else 1 for ins in val_data]
elif prompt == "mq":
    pred_list = [0 if "no" in ins['response'].lower() else 1 for ins in val_data]
evaluate(label_list, pred_list, show=True)

### Linear probing

In [ ]:
model = LogisticRegression()
model.fit(x_train, y_train)

y_pred = model.predict_proba(x_val)[:, 1]
_, _, _, auroc = evaluate(y_val, y_pred, show=True)